* Used llama 3 8B-instruct
* Good at reading checked forms
* Loading takes much time in A6000 GPU(nearly 30 mins)
* Can process one image at a time
* Can not able to handle complex handwritten texts 
* Produces Wrong answer if the answer is not in the image

In [ ]:
%pip install transformers einops xformers

In [2]:
import torch
from PIL import Image
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_PATH = "THUDM/cogvlm2-llama3-chat-19B"
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
TORCH_TYPE = torch.bfloat16 if torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8 else torch.float16

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=TORCH_TYPE,
    trust_remote_code=True,
).to(DEVICE).eval()




model-00005-of-00008.safetensors:  95%|#########4| 4.75G/5.00G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [ ]:
text_only_template = "A chat between a curious user and an artificial intelligence assistant. The assistant provides helpful, detailed, and polite answers to the user's questions based on the image. If the assistant cannot find the accurate answer from the image, the assistant responds with 'Answer Not Found.' USER: {} ASSISTANT:"

image_path1 = '/home/redacted_pdf_pg_no_69.png'
image_path2 = '/home/redacted_pdf_pg_no_28.png'
image_path3 = "/home/Screenshot 2024-11-15 124654.png"
image_path4 = "/home/Screenshot 2024-11-18 164554.png"
image_path5 = "/home/Screenshot 2024-11-18 165115.png"

image1 = Image.open(image_path1).convert('RGB')
image2 = Image.open(image_path2).convert('RGB')
image3 = Image.open(image_path3).convert('RGB')
image4 = Image.open(image_path4).convert('RGB')
image5 = Image.open(image_path5).convert("RGB")
history = []


In [ ]:
query = input("Human:")


input_by_model = model.build_conversation_input_ids(
        tokenizer,
        query=query,
        history=history,
        images=[image5],
        template_version='chat'
    )
inputs = {
    'input_ids': input_by_model['input_ids'].unsqueeze(0).to(DEVICE),
    'token_type_ids': input_by_model['token_type_ids'].unsqueeze(0).to(DEVICE),
    'attention_mask': input_by_model['attention_mask'].unsqueeze(0).to(DEVICE),
    'images': [[input_by_model['images'][0].to(DEVICE).to(TORCH_TYPE)]] 
}
gen_kwargs = {
    "max_new_tokens": 2048,
    "pad_token_id": 128002,  
}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    response = tokenizer.decode(outputs[0])
    response = response.split("<|end_of_text|>")[0]
    print("\nCogVLM2:", response)
history.append((query, response))